In [1]:
%run my_import

uri = socket.gethostname()+".local:5555"
#uri = "localhost:5555"
#uri = "nuc-i7.local:5555"
#uri = "com-exp.local:5555"
#uri = "ipc3-xeno.local:5555"
#uri = "192.168.99.10:5555"
#uri = "mirror-ecat-srv.local:5555"

io = set_uri(uri)
io.debug = False


/home/amargan/work/anaconda3/envs/repl/bin/python
new uri amargan-ryzen.local:5555


In [2]:
scan_ids = reply_cmd(master_cmd_get_slave_descr).keys()
ids=list(scan_ids)
print(ids)
#assert(iD in ids)

[1]


In [5]:
iD = 1
sdo_objd = reply_cmd(SdoInfo(u'SDO_OBJD').set_bid(iD))
sdo_names = reply_cmd(SdoInfo(u'SDO_NAME').set_bid(iD))
sdo_dict = reply_cmd(SdoCmd(rd_sdo=sdo_names,wr_sdo={}).set_bid(iD))

In [6]:
sf =sdo_filter(['m3_fw_ver','c28_fw_ver','hw_config_low','hw_config_high'],sdo_dict)
print(sf)
hex( (sf['hw_config_high']<<16) + sf['hw_config_low'] )

{'m3_fw_ver': 22121314, 'c28_fw_ver': 21122115, 'hw_config_low': 21527, 'hw_config_high': 21248}


'0x53005417'

In [9]:
# release all ids brakes using aux_pdo command
from struct import pack, unpack
brk_rel = unpack('f',pack('i',1))[0]
for _id in ids :
    #reply_cmd(SdoCmd(rd_sdo=[],wr_sdo={'master_cmd_control': brk_rel}).set_bid(_id))
    #time.sleep(0.2)
    ret = reply_cmd(SdoCmd(rd_sdo=['master_cmd_status'],wr_sdo={}).set_bid(iD))['master_cmd_status']
    print(_id, ret)

1 0.0


In [10]:
# engage all ids brakes using aux_pdo command
brk_eng = unpack('f',pack('i',2))[0]
for _id in ids :
    reply_cmd(SdoCmd(rd_sdo=[],wr_sdo={'master_cmd_control': brk_eng}).set_bid(_id))
    time.sleep(0.2)
    ret = reply_cmd(SdoCmd(rd_sdo=['master_cmd_status'],wr_sdo={}).set_bid(_id))['master_cmd_status']
    print(_id, ret)

1 0.0


In [7]:
sdo = ['torque','motor_pos','link_pos','Min_pos','Max_pos','Max_cur']
sdo = ['link_pos','fault','linkEncBadReadPPM','absLinkEncErr']
for i in range(2):
    G = io.doit4ids([iD],SdoCmd(rd_sdo=sdo,wr_sdo={}))
    #[g for g in G ]
    for g in G :
        print(g['msg'])
    time.sleep(0.2)

{absLinkEncErr: 0, fault: 0, linkEncBadReadPPM: 0.000000, link_pos: 0.036181}
{absLinkEncErr: 0, fault: 0, linkEncBadReadPPM: 0.000000, link_pos: 0.036175}


In [8]:
io.doit(ctrl_cmd_start().set_bid(iD).set_value(0x3B))

{'type': 1, 'cmd_type': 2, 'msg': 'Ok !!!'}

In [ ]:
io.doit(ctrl_cmd_stop.set_bid(iD))

In [ ]:
io.doit(ctrl_cmd_start().set_bid(iD).set_value(0xD4))

In [ ]:
io.debug=0
gains = {}
gains['type'] = "IMPEDANCE"
gains['kp'] = 250
gains['kd'] = 5
#gains['ki'] = 0
gains['kkp'] = 2
gains['kkd'] = 0.001
gains['kki'] = 0.08
#io.doit(ctrl_cmd_start().set_bid(iD).set_value(0xD4).set_gains(*gains.values()))
io.doit(ctrl_cmd_start().set_bid(iD).set_value(0xD4).set_gains(u'IMPEDANCE',2000,10,2,0.001,0))
#io.doit(ctrl_cmd_start().set_bid(iD).set_value(0xD4).set_gains(u'IMPEDANCE',0,0,5,0.001,0.08))

In [ ]:
io.doit(ctrl_cmd_stop.set_bid(iD))

In [15]:
for _id in ids :
    ctrl_status_cmd_str('RELEASE_BRAKE', _id)

0x78bd


In [12]:
for _id in ids :
    ctrl_status_cmd_str('ENGAGE_BRAKE', _id)

0x78db


In [13]:
ctrl_status_cmd_str('FAN_ON', iD)

0xaa26


False

In [14]:
ctrl_status_cmd_str('FAN_OFF', iD)

0xaa62


False

In [ ]:
io.doit(SdoCmd(rd_sdo=[],wr_sdo={'Max_cur': 10}).set_bid(iD))

In [ ]:
io.doit(flash_cmd_save_flash.set_bid(iD))

In [11]:
sdo_names = reply_cmd(SdoInfo(u'SDO_NAME').set_bid(123))
sdo_dict = reply_cmd(SdoCmd(rd_sdo=sdo_names,wr_sdo={}).set_bid(123))

In [26]:
io.doit(SdoCmd(rd_sdo=[],wr_sdo={'hw_config_low': 0x6106,'hw_config_high': 0x1100, 'motor_velocity_constant': 122}).set_bid(123))

{'type': 1, 'cmd_type': 7, 'msg': '{}'}

In [24]:
io.doit(flash_cmd_save_flash.set_bid(123))

{'type': 1, 'cmd_type': 3, 'msg': 'Ok !!!'}

In [27]:
sdo_dict

{'Calib_angle': 3.14159,
 'CurrGainI': 0.01,
 'CurrGainP': 0.2,
 'DOB_filterFrequencyHz': 10.0,
 'DacChA': 1,
 'DacChB': 15,
 'Direct_ref': 0.0,
 'Enc_offset': -0.414693,
 'Joint_robot_id': 123,
 'Max_cur': 4.0,
 'Max_pos': 2.891593,
 'Max_tor': 147.0,
 'Max_vel': 3.86,
 'Min_pos': -2.891593,
 'Motor_el_ph_angle': 5.3,
 'Motor_gear_ratio': 160,
 'Serial_Number_A': 128,
 'Torsion_bar_stiff': 5520.924316,
 'absDeflEncErr': 0,
 'absDeflEncWarn': 188389,
 'absLinkEncErr': 0,
 'absLinkEncWarn': 0,
 'absMotorEncErr': 0,
 'absMotorEncWarn': 0,
 'backEmfCompensation': 0.8,
 'board_faults': 385,
 'board_temp': 41.857533,
 'board_temp_fb': 41.852642,
 'c28_fw_ver': 21122115,
 'crossTermCCGain': 0.0,
 'ctrl_status_cmd': 90,
 'ctrl_status_cmd_ack': 30810,
 'deflEncBadReadPPM': 0.0,
 'deflEncRoughRead': 0,
 'defl_enc_err': 0.0,
 'defl_enc_warn': 0.0,
 'deflection_encoder': 0.0,
 'directTorqueFeedbackGain': 1.0,
 'fault': 385,
 'faultPowerOff': 0,
 'fault_ack': 0,
 'flash_params_cmd': 0,
 'flash_par